In [1]:
import matplotlib.pyplot as plt
import torch
import ehc_sn as ehc

In [2]:
from ipywidgets import interact, IntSlider, FloatSlider
from functools import partial

IntSlider = partial(IntSlider, continuous_update=False)
FloatSlider = partial(FloatSlider, continuous_update=False)

In [3]:
import tomllib as toml

with open("configurations/experiment_20250312.toml", "rb") as f:
    data = toml.load(f)

In [4]:
def plot_raster(spikes, title):
    plt.figure(figsize=(10, 5))
    for neuron_idx in range(spikes.shape[1]):
        spike_times = torch.nonzero(spikes[:, neuron_idx]).squeeze()
        plt.scatter(spike_times, neuron_idx * torch.ones_like(spike_times), s=1)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Neuron Index")
    plt.xlim(0, spikes.shape[0])
    plt.ylim(0, spikes.shape[1])
    plt.show()

In [5]:
model = ehc.EIModel(p=ehc.EIParameters.model_validate(data))
model

EIModel(
  (excitatory): LIFRefracCell(p=LIFRefracParameters(lif=LIFParameters(tau_syn_inv=tensor(2., device='cuda:0'), tau_mem_inv=tensor(0.5000, device='cuda:0'), v_leak=tensor(-60., device='cuda:0'), v_th=tensor(-50., device='cuda:0'), v_reset=tensor(-60., device='cuda:0'), method='super', alpha=tensor(0.5000, device='cuda:0')), rho_reset=tensor(2., device='cuda:0')), dt=0.001)
  (inihibitory): LIFRefracCell(p=LIFRefracParameters(lif=LIFParameters(tau_syn_inv=tensor(1., device='cuda:0'), tau_mem_inv=tensor(0.5000, device='cuda:0'), v_leak=tensor(-60., device='cuda:0'), v_th=tensor(-50., device='cuda:0'), v_reset=tensor(-60., device='cuda:0'), method='super', alpha=tensor(0.5000, device='cuda:0')), rho_reset=tensor(2., device='cuda:0')), dt=0.001)
)

In [7]:
@interact(
    current=FloatSlider(min=0, max=10, step=0.1, value=2.4),
    t1=IntSlider(min=0, max=1000, step=1, value=100),
    t2=IntSlider(min=0, max=1000, step=1, value=300),
)
def experiment(current, t1, t2):
    model.reset()
    input_current = torch.zeros(1000, model.populations.n_exc).to(ehc.device)
    input_current[t1:t2, :] = current  # Apply current to the first neuron
    outputs = model(input_current)
    plot_raster(outputs.to("cpu"), "Balanced state spikes")

interactive(children=(FloatSlider(value=2.4, continuous_update=False, description='current', max=10.0), IntSli…